**Modern Portfolio Theory**


In [6]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [169]:
import numpy as np
import datetime as dt
import yfinance as yf
import pandas as pd
import scipy as scy
from scipy import optimize as opt
import plotly.graph_objects as go

In [54]:
apple = yf.Ticker("AAPL")
tesla = yf.Ticker("TSLA")
meta = yf.Ticker("META")

apple = (yf.Ticker("AAPL")).history(period = "1y")
tesla = (yf.Ticker("TSLA")).history(period = "1y")
meta = (yf.Ticker("META")).history(period = "1y")

stockData = pd.DataFrame(zip(apple['Close'],tesla['Close'],meta['Close']), columns =["AAPL","TSLA","META"])
returns = stockData.pct_change()
mean_returns = returns.mean()
cov_matrix = returns.cov()

In [126]:
def ptf_performance(weights,mean_returns,cov_matrix):
  returns = np.sum(mean_returns * weights) * 252
  st_dev = np.sqrt(np.dot(weights.T,np.dot(cov_matrix,weights))) * np.sqrt(252)
  return returns,st_dev

weights = np.array([0.3,0.3,0.4])
mean_returns, cov_matrix  
returns,  st_dev = ptf_performance(weights, mean_returns, cov_matrix)

print(round(returns*100,2),round(st_dev*100,2))


-19.3 39.43


In [127]:
def neg_sharperatio(weigths,mean_returns,cov_matrix,rfrate= 0):
   preturns, pst_dev = ptf_performance(weights, mean_returns, cov_matrix)
   return - (preturns - rfrate)/pst_dev



def maxSR(mean_returns,cov_matrix, rfrate = 0, constr_set = (0,1)):
  "We want to minimize the negative SR, by altering the weights of the ptf"
  num_assets = len(mean_returns)
  args = (mean_returns,cov_matrix, rfrate)
  constraints = ({'type':'eq','fun':lambda x:np.sum(x) - 1}) ##sum of weigths has to be 1
  bound = constr_set
  bounds = tuple(bound for asset in range(num_assets))
  result = opt.minimize(neg_sharperatio, num_assets*[1/num_assets], args = args,
                       method = 'SLSQP',bounds = bounds,constraints = constraints) ## neg_sharperatio is what we try to minimize, the second argument is the pmt we need to minimize
  return result                     


#My_result = maxSR(mean_returns,cov_matrix)
#maxSR,max_weights =  My_result['fun'],My_result['x']
#print(maxSR,max_weights)

**Minimum variance ptf**

In [164]:
def ptf_variance(weights,mean_returns,cov_matrix):
  return ptf_performance(weights,mean_returns,cov_matrix)[1]


def minimize_var(mean_returns,cov_matrix, rfrate = 0, constr_set = (0,1)):
  "Minimize the ptf variance by altering the weights/allocation of assets in ptf"
  num_assets = len(mean_returns)
  args = (mean_returns,cov_matrix)
  constraints = ({'type':'eq','fun':lambda x:np.sum(x) - 1}) ##sum of weigths has to be 1
  bound = constr_set
  bounds = tuple(bound for asset in range(num_assets))
  result = opt.minimize(ptf_variance, num_assets*[1/num_assets], args = args,
                       method = 'SLSQP',bounds = bounds,constraints = constraints)
  return result 


#My_minvar_result = minimize_var(mean_returns,cov_matrix)
#minimize_var,minvar_weights =  My_minvar_result['fun'],My_minvar_result['x']
#print(minimize_var,minvar_weights)  


In [130]:
maxSR(mean_returns,cov_matrix)

     fun: 0.48959489890811525
     jac: array([0., 0., 0.])
 message: 'Optimization terminated successfully.'
    nfev: 5
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([0.33333333, 0.33333333, 0.33333333])

In [188]:
def calculated_results(mean_ret,cov_mat,rfrate=0, constr_set=(0,1)):
     """Read in mean, cov matrix, ando other financial information
        Output, Max SR, min volatility, efficient frontier  """
     ## max sharpe ratio ptf   
     max_SR_ptf = maxSR(mean_ret, cov_mat)
     max_SR_ret,max_SR_sdev = ptf_performance(max_SR_ptf['x'],mean_ret,cov_mat)   
     max_SR_allocation = pd.DataFrame(max_SR_ptf['x'],index= mean_ret.index,columns = ['Allocation'])
     max_SR_allocation['Allocation'] = [round(i*100,0) for i in max_SR_allocation['Allocation']]
     
     ## min volatility ptf
     min_vol_ptf = minimize_var(mean_ret, cov_mat)
     min_vol_ret,min_vol_sdev = ptf_performance(min_vol_ptf['x'],mean_ret,cov_mat)
     min_vol_allocation = pd.DataFrame(min_vol_ptf['x'],index= mean_ret.index,columns = ['Allocation'])
     min_vol_allocation['Allocation'] = [round(i*100,0) for i in min_vol_allocation['Allocation']]


     ## Efficient frontier
     target_returns = np.linspace(min_vol_ret,max_SR_ret,20)
     efficient_list = []
     for target in target_returns:
       efficient_list.append(efficient_opt(mean_returns, cov_matrix,target)['fun'])

     max_SR_ret,max_SR_sdev = round(max_SR_ret*100,2), round(max_SR_sdev*100,2)  
     min_vol_ret,min_vol_sdev = round(min_vol_ret*100,2), round(min_vol_sdev*100,2)

     return max_SR_ret,max_SR_sdev,max_SR_allocation, min_vol_ret, min_vol_sdev, min_vol_allocation, efficient_list, target_returns

     

calculated_results(mean_returns,cov_matrix)  



(-14.11, 39.16,       Allocation
 AAPL        33.0
 TSLA        33.0
 META        33.0, 3.06, 29.89,       Allocation
 AAPL        98.0
 TSLA         0.0
 META         2.0, [0.2989317671173765,
  0.2989721984836951,
  0.2991123462214718,
  0.2993520702810004,
  0.29969113170916495,
  0.300129193831281,
  0.3006658239238233,
  0.30130049533233616,
  0.3020325899967095,
  0.302861401440145,
  0.3037861380565486,
  0.30480592669098017,
  0.3059198169269494,
  0.3071267847912833,
  0.30842573761435615,
  0.3098155183051615,
  0.31129491053534736,
  0.3128626429429449,
  0.3145173945534887,
  0.3162577995572117], array([ 0.03061043,  0.02157196,  0.0125335 ,  0.00349503, -0.00554344,
        -0.01458191, -0.02362038, -0.03265884, -0.04169731, -0.05073578,
        -0.05977425, -0.06881272, -0.07785118, -0.08688965, -0.09592812,
        -0.10496659, -0.11400505, -0.12304352, -0.13208199, -0.14112046]))

In [166]:
def efficient_opt(mean_returns,cov_matrix,return_target,constr_set=(0,1)):
  """ For each return_target, we want to optimise the ptf for min variance """
  num_assets = len(mean_returns)
  args = (mean_returns, cov_matrix)
  ##def ptf_return(weights):
      ##return ptf_performance(weights,mean_returns,cov_matrix)[0]
  constraints = ({'type': 'eq', 'fun': lambda x: ptf_return(x,mean_returns,cov_matrix) - return_target},
                 {'type':'eq','fun':lambda x:np.sum(x) - 1})
  bound = constr_set
  bounds = tuple(bound for asset in range(num_assets))
  eff_opt = opt.minimize(ptf_variance, num_assets*[1/num_assets], args = args,
                       method = 'SLSQP',bounds = bounds,constraints = constraints)
  return eff_opt
  


efficient_opt(mean_returns,cov_matrix,0.06)
  

     fun: 0.31185312571475415
     jac: array([0.29420054, 0.45030959, 0.29111474])
 message: 'Optimization terminated successfully.'
    nfev: 20
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([0.88692145, 0.11307855, 0.        ])

In [197]:
def EF_graph(mean_returns, cov_matrix, rfrate=0, constr_set=(0,1)):
  """Return a graph plottung the min vol, max sr efficient frontier """
  max_SR_ret,max_SR_sdev,max_SR_allocation, min_vol_ret, min_vol_sdev, min_vol_allocation, efficient_list, target_returns = calculated_results(mean_returns,cov_matrix,rfrate, constr_set)

  ## Max SR
  max_sp = go.Scatter(
      name = 'Maximum Sharpe Ratio',
      mode = 'markers',
      x = [max_SR_sdev],
      y =[max_SR_ret],
      marker = dict(color='red',size=14,line=dict(width=3,color='black'))
  )

  ## Min vol
  min_vol = go.Scatter(
      name = 'Minimum Volatility',
      mode = 'markers',
      x = [min_vol_sdev],
      y =[min_vol_ret],
      marker = dict(color='green',size=14,line=dict(width=3,color='black'))
  )

  ## Efficient frontier
  ef_curve = go.Scatter(
      name = 'Efficient Frontier',
      mode = 'lines',
      x = [round(ef_std*100,2)for ef_std in efficient_list],
      y =[round(target*100,2) for target in target_returns],
      line = dict(color='black',width=4,dash ='dashdot')
  )

  data = [max_sp, min_vol, ef_curve]
  
  layout = go.Layout(
      title = 'Portfolio Optimisation with the efficient frontier',
      yaxis = dict(title ='Annualized Return (%)'),
      xaxis = dict(title = 'Annualized volatility (%)'),
      showlegend=True,
      legend = dict(
            x =0.75, y = 0, traceorder= 'normal',
            bgcolor = '#E2E2E2',
            bordercolor = 'black',
            borderwidth =2),
      width = 800,
      height = 600)  

  fig = go.Figure(data = data, layout = layout)
  return fig.show()

  
  


EF_graph(mean_returns, cov_matrix)
  



